3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env



In [2]:
def acquire():
    # CSV file available?
    filename = 'curriculum.csv'
    # Path
    if os.path.exists(filename):
        
        return pd.read_csv(filename)
    
    else:
        url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
        query = """SELECT * FROM logs LEFT JOIN cohorts ON logs.user_id = cohorts.id"""
        df = pd.read_sql(query, url)
        
        df.to_csv(filename, index=False)
        
        return df

In [3]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = """SELECT * FROM logs LEFT JOIN cohorts ON logs.user_id = cohorts.id"""
df = pd.read_sql(query, url)
# df.to_csv(cirriculum.csv, index=False)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0


### Looking throuhg and understanding the data 

In [4]:

df.path.value_counts()

/                                                               50313
search/search_index.json                                        19519
javascript-i                                                    18983
toc                                                             18297
java-iii                                                        13733
                                                                ...  
javascript/loops                                                    1
content/control-structures-ii                                       1
javascript-ii/promisesdfghjkjhgfs                                   1
app                                                                 1
appendix/professional-development/post-interview-review-form        1
Name: path, Length: 2313, dtype: int64

In [6]:
df.program_id.value_counts()

1.0    37794
2.0    31980
3.0     2458
4.0     1507
Name: program_id, dtype: int64

In [7]:
df.shape

(900223, 15)

In [8]:
df.user_id.value_counts

<bound method IndexOpsMixin.value_counts of 0           1
1           1
2           1
3           1
4           2
         ... 
900218     64
900219     64
900220     64
900221    744
900222     64
Name: user_id, Length: 900223, dtype: int64>

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          73739 non-null   float64
 7   name        73739 non-null   object 
 8   slack       73739 non-null   object 
 9   start_date  73739 non-null   object 
 10  end_date    73739 non-null   object 
 11  created_at  73739 non-null   object 
 12  updated_at  73739 non-null   object 
 13  deleted_at  0 non-null       object 
 14  program_id  73739 non-null   float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [ ]:
#im removing staff 

In [13]:
df.name.value_counts()

Ike                                       17913
Fortuna                                   12329
Arches                                     7404
Redwood                                    2263
Wrangell                                   1809
Glacier                                    1803
Everglades                                 1701
Carlsbad                                   1562
Luna                                       1549
Badlands                                   1541
Quincy                                     1533
Pinnacles                                  1532
Marco                                      1459
Franklin                                   1408
Ada                                        1397
Neptune                                    1264
Andromeda                                  1004
Ulysses                                     969
Balboa                                      922
Zion                                        894
Joshua                                  

In [11]:
df = df.loc[df["name"] != 'Staff']

In [12]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0


In [21]:
df= df.drop(columns= ['id', 'updated_at', 'deleted_at'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899941 entries, 0 to 900222
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        899941 non-null  datetime64[ns]
 1   time        899941 non-null  object        
 2   path        899940 non-null  object        
 3   user_id     899941 non-null  int64         
 4   cohort_id   847048 non-null  float64       
 5   ip          899941 non-null  object        
 6   name        73457 non-null   object        
 7   slack       73457 non-null   object        
 8   start_date  73457 non-null   object        
 9   end_date    73457 non-null   object        
 10  created_at  73457 non-null   object        
 11  program_id  73457 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 89.3+ MB


In [23]:
# convert dates to date time type

df['date'] = pd.to_datetime(df['date'])
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899941 entries, 0 to 900222
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        899941 non-null  datetime64[ns]
 1   time        899941 non-null  object        
 2   path        899940 non-null  object        
 3   user_id     899941 non-null  int64         
 4   cohort_id   847048 non-null  float64       
 5   ip          899941 non-null  object        
 6   name        73457 non-null   object        
 7   slack       73457 non-null   object        
 8   start_date  73457 non-null   datetime64[ns]
 9   end_date    73457 non-null   datetime64[ns]
 10  created_at  73457 non-null   object        
 11  program_id  73457 non-null   float64       
dtypes: datetime64[ns](3), float64(2), int64(1), object(6)
memory usage: 89.3+ MB
